# Exploring an ItemCollection

Let's load an `ItemCollection` of Landsat 8 scenes from the Planetary Computer STAC and preview them with `stacmap.explore`.

In [1]:
import stacmap
import pystac_client

In [2]:
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

aoi = {"type": "Polygon", "coordinates": [[
    [-112.2068359375, 42.41050500280784],
    [-112.2068359375, 39.25270077697571],
    [-107.9880859375, 39.25270077697571],
    [-107.9880859375, 42.41050500280784],
    [-112.2068359375, 42.41050500280784]
]]}

items = catalog.search(
    collections=["landsat-8-c2-l2"],
    intersects=aoi,
    datetime="2019-06-01/2019-06-10"
).get_all_items()

Calling `stacmap.explore` with a STAC Collection* will preview the item footprints on an interactive map. Hover over each footprint to see its STAC metadata.

*Currently supported STAC-like types are: `pystac.Catalog`, `pystac.ItemCollection`, `pystac.Item`, collection dictionaries, item dictionaries, and lists of item dictionaries.

In [5]:
stacmap.explore(items)

## Color-Coding

Footprints can be color-coded based on STAC properties using the `prop` parameter. The default colormap can be replaced with any [colorbrewer](https://github.com/axismaps/colorbrewer/) name using `cmap`. Additional color maps are supported if `matplotlib` is installed.

### Continuous Properties

By default, numeric properties are colored continuously and given continuous legends. Color map bounds are calculated automatically, but can be overriden by providing `vmin` and `vmax` arguments.

In [15]:
stacmap.explore(items, prop="view:sun_elevation", cmap="Reds")

### Categorical Properties

Non-numeric properties are treated categorically by default, with a color assigned to each unique value. You can force a numerical field to plot categorically using `categorical`.

In [13]:
stacmap.explore(items, prop="proj:epsg", categorical=True)

## Search Extents

Pass a bounding box to `bbox` or a GeoJSON to `intersects` to overlay your search bounds.

In [14]:
stacmap.explore(items, prop="landsat:wrs_path", intersects=aoi)

## Custom Styling

Visualization styles can be customized using `style_kwds`, `highlight_kwds`, and `bounds_kwds`. `stacmap.explore` follows the `geopandas.GeoDataFrame.explore` implementation for these styling options, so check out [their docs](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html#geopandas-geodataframe-explore) to get a list of supported options. You can also customize the Folium basemap using the `tiles` parameter.

In [12]:
stacmap.explore(
    items, tiles="CartoDB dark_matter", intersects=aoi,
    style_kwds={"color": "#c44500", "weight": 4, "fillOpacity": 0.1}, 
    highlight_kwds={"fillOpacity": 0.9},
    bounds_kwds={"color": "#ffe814", "opacity": 1.0}
)

## Custom Metadata

### Filtering Properties

By default, `stacmap.explore` gives a tooltip that displays the item ID and all STAC properties on-hover. Metadata properties can be filtered by passing a list of properties to include to `fields` or by passing one or more STAC extension prefixes to `extensions`. Note that with the second option, base STAC properties are always included.

In [21]:
stacmap.explore(items, extensions=["view", "eo"])

### More Metadata Options

- Set `shared_props=True` to only include properties that are shared by all items. Otherwise, if items contain different properties then missing properties will be filled with `None`.
- Set `add_id=False` to avoid automatically adding each item's STAC ID to the metadata properties.

## Using Existing Maps

`stacmap.explore` returns a Folium map. You can add multiple STAC collections by passing an existing map to the `m` parameter. Pass a `name` parameter with each STAC collection to allow layer control. Below, we'll split the Landsat items into two sub-groups and plot each separately on the same map.

In [26]:
m = stacmap.explore(items[:5], name="Landsat #1", prop="proj:epsg", categorical=True)
stacmap.explore(items[5:], m=m, name="Landsat #2", prop="eo:cloud_cover", cmap="BrBG", fields=["eo:cloud_cover", "instruments"])
m

### Thumbnails

Some STAC items have links to thumbnail assets. If they are available, you can overlay thumbnail images by setting `thumbnails=True`. Note that catalogs like Planetary Computer may need to be signed to access thumbnails.

In [10]:
import planetary_computer

stacmap.explore(planetary_computer.sign(items[0]), thumbnails=True, style_kwds={"fill": False})